# UAS Computer Vision
### Nama : Leo Christopher Coo
### NIM  : 2201808125
<br><br>


In [1]:
# only run this in google colab to upload data
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [2]:
# only run this in google colab to unzip data
!unzip "COMP6588_Information Retrieval_REGULER_UAS.zip"

### Install Sastrawi untuk stemmer dan stopwords

In [3]:
!pip install Sastrawi

# No. 1

### Library yang digunakan

In [4]:
import pandas as pd
import numpy as np 
import re # impor modul regular expression
import os
from string import punctuation
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory# import StemmerFactory class
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import nltk
from keras.utils.np_utils import to_categorical
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Load CSV File and txt File

In [5]:
path = os.getcwd()+ ("/COMP6588_Information Retrieval_REGULER_UAS/COMP6588_Information Retrieval_REGULER_UAS - Dataset/")
# Read raw data
df_sentiment = pd.read_csv(path + 'sentiment.csv', sep=";", header=0)

df_hs = pd.read_csv(path+'hatespeech.txt',sep="\t", header=0, engine='python')

### Text Preprocessing untuk Dataset Sentiment dan hatespeech

### Menghapus semua Hyperlink

In [6]:
df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: re.sub(r"http\S+",'',x))
df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: re.sub(r"http\S+",'',x))

### Menghapus semua @"username"

In [7]:
df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: (x.replace(':','')))
df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: re.sub('@[A-Za-z0-9]+','', x))

df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: (x.replace(':','')))
df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: re.sub('@[A-Za-z0-9]+','', x))


### Menghapus \n dan \u

In [8]:
df_sentiment["Tweets"] = df_sentiment["Tweets"].replace(r'\\n','',regex =True)
df_sentiment["Tweets"] = df_sentiment["Tweets"].replace(r'\\u','',regex =True)

df_hs["Tweet"] = df_hs["Tweet"].replace(r'\\n','',regex =True)
df_hs["Tweet"] = df_hs["Tweet"].replace(r'\\u','',regex =True)

### Menghapus tanda baca

In [9]:
df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: "".join([c for c in x if c not in punctuation]))
df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: "".join([c for c in x if c not in punctuation]))

df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: (x.replace('_','')))
df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: (x.replace('_','')))


### Mengubah semua huruf menjadi huruf kecil

In [10]:
df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: str.lower(x))
df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: str.lower(x))

### Menghapus semua angka

In [11]:
df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: "".join(re.sub('[0-9]+', '', x)))
df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: "".join(re.sub('[0-9]+', '', x)))

### Menghapus Stopwords

In [12]:
stopwords =stopwords.words('indonesian')

### tambah stopword (yg ,sprt,...)
newStop = ['rt']
stopwords.extend(newStop)

In [13]:
def remove_stopwords(s):
    s=' '.join(word for word in s.split()if word not in stopwords)
    return s

df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: remove_stopwords(x))
df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: remove_stopwords(x))

### Menganti Label dari string menjadi numerik

In [14]:
df_sentiment["Label"] = df_sentiment["Label"].apply(lambda x: 1 if x == 'negatif' else 0)
df_hs["Label"] = df_hs["Label"].apply(lambda x: 1 if x == 'HS' else 0)

### Stemming, Mengubah kata menjadi bentuk dasarnya

In [15]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df_sentiment["Tweets"] = df_sentiment["Tweets"].apply(lambda x: stemmer.stem(x))
df_hs["Tweet"] = df_hs["Tweet"].apply(lambda x: stemmer.stem(x))

### Tokenize 

In [16]:
max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)

# sentiment
tok.fit_on_texts(df_sentiment["Tweets"].values)
X_sentiment  = tok.texts_to_sequences(df_sentiment["Tweets"].values)
X_sentiment = sequence.pad_sequences(X_sentiment)

# hatespeech
tok.fit_on_texts(df_hs["Tweet"].values)
X_hs = tok.texts_to_sequences(df_hs["Tweet"].values)
X_hs = sequence.pad_sequences(X_hs)

### RNN

In [17]:
def RNN(X):
  embed_dim = 128
  lstm_out = 196

  model = Sequential()
  model.add(Embedding(max_words, embed_dim))
  model.add(SpatialDropout1D(0.4))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='sigmoid'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  print(model.summary())

  return model


In [18]:
model_sentiment = RNN(X_sentiment)
model_hs = RNN(X_hs)
model_hs.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         128000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, None, 128)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 383,194
Trainable params: 383,194
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         128000    
_______________________

### Membuat data sentiment dan hatespeech menjadi  Training dan Test Dataset

In [19]:
Y_sentiment = pd.get_dummies(df_sentiment['Label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X_sentiment,Y_sentiment, test_size = 0.2, random_state = 0)

Y_sentiment2 = pd.get_dummies(df_hs['Label']).values
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X_hs,Y_sentiment2, test_size = 0.2, random_state = 0)

### Training

### Sentiment

In [20]:
model_sentiment.fit(X_train,Y_train,batch_size=10,epochs=10,validation_split=0.2)

Epoch 1/10
97/97 [==============================] - 14s 118ms/step - loss: 0.6713 - accuracy: 0.5948 - val_loss: 0.4501 - val_accuracy: 0.8257
Epoch 2/10
97/97 [==============================] - 10s 107ms/step - loss: 0.4010 - accuracy: 0.8210 - val_loss: 0.3975 - val_accuracy: 0.8091
Epoch 3/10
97/97 [==============================] - 10s 103ms/step - loss: 0.2945 - accuracy: 0.8773 - val_loss: 0.4110 - val_accuracy: 0.7884
Epoch 4/10
97/97 [==============================] - 10s 103ms/step - loss: 0.2478 - accuracy: 0.9181 - val_loss: 0.4228 - val_accuracy: 0.7967
Epoch 5/10
97/97 [==============================] - 10s 104ms/step - loss: 0.2089 - accuracy: 0.9197 - val_loss: 0.4625 - val_accuracy: 0.7884
Epoch 6/10
97/97 [==============================] - 10s 106ms/step - loss: 0.2073 - accuracy: 0.9182 - val_loss: 0.5333 - val_accuracy: 0.7842
Epoch 7/10
97/97 [==============================] - 10s 106ms/step - loss: 0.1576 - accuracy: 0.9387 - val_loss: 0.5400 - val_accuracy: 0.7759

In [21]:
model_hs.fit(X_train2,Y_train2,batch_size=10,epochs=10,validation_split=0.2)

Epoch 1/10
46/46 [==============================] - 9s 134ms/step - loss: 0.6341 - accuracy: 0.6501 - val_loss: 0.6025 - val_accuracy: 0.7018
Epoch 2/10
46/46 [==============================] - 6s 130ms/step - loss: 0.4980 - accuracy: 0.7619 - val_loss: 0.3686 - val_accuracy: 0.8684
Epoch 3/10
46/46 [==============================] - 6s 130ms/step - loss: 0.2737 - accuracy: 0.9146 - val_loss: 0.3427 - val_accuracy: 0.8772
Epoch 4/10
46/46 [==============================] - 6s 127ms/step - loss: 0.1163 - accuracy: 0.9595 - val_loss: 0.4133 - val_accuracy: 0.8421
Epoch 5/10
46/46 [==============================] - 6s 120ms/step - loss: 0.0788 - accuracy: 0.9774 - val_loss: 0.4166 - val_accuracy: 0.8421
Epoch 6/10
46/46 [==============================] - 6s 129ms/step - loss: 0.0528 - accuracy: 0.9799 - val_loss: 0.4865 - val_accuracy: 0.8509
Epoch 7/10
46/46 [==============================] - 6s 126ms/step - loss: 0.0626 - accuracy: 0.9835 - val_loss: 0.4930 - val_accuracy: 0.8421
Epoch 

### Evaluasi model sentiment

In [22]:
accr = model_sentiment.evaluate(X_test,Y_test)

10/10 [==============================] - 0s 16ms/step - loss: 0.6709 - accuracy: 0.7781


### Evaluasi model hatespeech

In [23]:
h_accr = model_hs.evaluate(X_test2,Y_test2)

5/5 [==============================] - 0s 18ms/step - loss: 0.8219 - accuracy: 0.7762


# No. 2 cross-inferencing

### Evaluasi model sentiment pada data hatespeech

In [24]:
s_accr_h = model_sentiment.evaluate(X_test2,Y_test2)

5/5 [==============================] - 0s 24ms/step - loss: 1.5917 - accuracy: 0.5245


### Prediksi Model Sentiment terhadap Data Hatespeech

In [ ]:
validation_size = 600

X_validate = X_test2[-validation_size:]
Y_validate = Y_test2[-validation_size:]
X_test1 = X_test2[:-validation_size]
Y_test1 = Y_test2[:-validation_size]

pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model_sentiment.predict(X_validate[x].reshape(1,X_test1.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1


In [26]:
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 73.68421052631578 %
neg_acc 38.372093023255815 %


### Evaluasi model hatespeech pada data sentiment

In [27]:
h_accr_s = model_hs.evaluate(X_test,Y_test)

10/10 [==============================] - 1s 22ms/step - loss: 2.5682 - accuracy: 0.5364


### Prediksi Model Hatespeech pada Data sentiment

In [ ]:
validation_size = 600

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model_sentiment.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1


In [29]:
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 76.58227848101265 %
neg_acc 79.16666666666666 %


### Pada Hasil yang didapatkan pada kedua model tersebut, adalah **Hate Speech cenderung Mempunyai Sentimen Negatif (Model Hate Speech dapat memprediksi Sentimen Negatif dengan tingkat keakuratan 79%), tetapi Sentimen Negatif cenderung tidak mepunyai Hate Speech (Model Sentiment memprediksi Hate Speech dengan tingkat keakuratan 38%)**.